In [56]:
import asyncio
import re
import os
import pandas as pd
import base64
from openai import AsyncOpenAI
from dotenv import load_dotenv
import io
from copy import deepcopy
# from evaluation.utils import code_to_image
import matplotlib.pyplot as plt
from loguru import logger # type: ignore
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential, stop_after_delay,
)


# Load environment variables
load_dotenv()

QUERY_EXPANSION_PROMPT='''According to the user query, expand and solidify the query into a step by step detailed instruction (or comment) on how to write python code to fulfill the user query's requirements. Import the appropriate libraries. Pinpoint the correct library functions to call and set each parameter in every function call accordingly.'''

SYSTEM_PROMPT_CODE_GEN='''You are a helpful assistant that generates Python code for data visualization and analysis using matplotlib, seaborn and pandas. Given a detailed instruction and data description, generate the appropriate code in the Markdwon format of ```python ...```. MUST FOLLOW THE FORMAT.'''

FEEDBACK_SYSTEM_PROMPT = '''Given a piece of code, a user query, and an image of the current plot, please determine whether the plot has faithfully followed the user query. Your task is to provide instruction to make sure the plot has strictly completed the requirements of the query. Please output a detailed step by step instruction on how to use python code to enhance the plot.'''

FEEDBACK_USER_PROMPT = '''Here is the code: [Code]:
"""
{{code}}
"""

Here is the user query: [Query]:
"""
{{query}}
"""

Carefully read and analyze the user query to understand the specific requirements. Examine the provided Python code to understand how the current plot is generated. Check if the code aligns with the user query in terms of data selection, plot type, and any specific customization. Look at the provided image of the plot. Assess the plot type, the data it represents, labels, titles, colors, and any other visual elements. Compare these elements with the requirements specified in the user query. Note any differences between the user query requirements and the current plot. Based on the identified discrepancies, provide step-by-step instructions on how to modify the Python code to meet the user query requirements. Suggest improvements for better visualization practices, such as clarity, readability, and aesthetics, while ensuring the primary focus is on meeting the user's specified requirements. If there is no base64 image due to an error(EX. "Error during Saving plot image: [Errno 2] No such file or directory: 'your_data.csv"), please check the error message and provide feedback based on the specific issue. The feedback should suggest appropriate actions to resolve the issue according to the error details.
'''

In [57]:
import tiktoken

def count_gpt4o_tokens(text: str) -> int:
    """
    GPT-4o 모델의 입력 텍스트에 대한 토큰 개수를 계산하는 함수
    """
    encoding = tiktoken.get_encoding("cl100k_base")  # GPT-4o에서 사용하는 인코딩
    tokens = encoding.encode(text)
    print(f"Token count: {len(tokens)}")
    # return len(tokens)

# 테스트 예제
sample_text = "Hello, how are you?"
token_count = count_gpt4o_tokens(sample_text)

Token count: 6


In [115]:
api_key = os.getenv("API_KEY")
base_url = os.getenv("BASE_URL")

client = AsyncOpenAI(
    api_key=api_key,
    base_url=base_url
)

model = 'gpt-4o-mini'
temperature = 0.2
# sample = [10, 20, 30, 40, 50]
sample = 50
dataset_path = r"..\..\dataset\matplotbench_data.csv"

dataset_df = pd.read_csv(dataset_path)
query_list = dataset_df.loc[sample,'simple_instruction']

In [116]:
img_save_path = f'./{sample}.png'
data_description='''There is no dataset provided.'''
QUERY_EXPANSION_PROMPT='''According to the user query, expand and solidify the query into a step by step detailed instruction (or comment) on how to write python code to fulfill the user query's requirements. Import the appropriate libraries. Pinpoint the correct library functions to call and set each parameter in every function call accordingly.'''

In [117]:
@retry(wait=wait_random_exponential(min=0.02, max=1), stop=(stop_after_delay(3) | stop_after_attempt(30)))
async def _call_openai_api(system_prompt, user_content):
    try:
        response = await client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_content}
            ],
            temperature=temperature
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"API call failed with error: {e}. Retrying...")
        raise e
    
async def _query_extension(nl_query):
    return await _call_openai_api(QUERY_EXPANSION_PROMPT, nl_query)

In [118]:
## Expansion
count_gpt4o_tokens(query_list)
output_extension = await _query_extension(query_list)

Token count: 246


In [119]:
# print(output_extension)
count_gpt4o_tokens(output_extension)

Token count: 997


In [120]:
user_content = f"""Detailed Instructions:
{output_extension}

Data Description:
Note that you must use the right csv data which is stated in "data_path". DO NOT MOCK DATA if data_path is provided!!!
If there are no data_path then just follow the Detailed Instructions. There might be an instruction about the data.
{data_description}

Please generate Python code using `matplotlib.pyplot` and 'seaborn' and 'pandas' to create the requested plot. Ensure the code is outputted within the Markdown format like ```python\n...```. 
You MUST follow the format and Don't savefig or save anything else.
"""
count_gpt4o_tokens(user_content)

Token count: 1117


In [121]:
img_save_path

'./50.png'

In [ ]:
def add_idx(path):
    parts = path.rsplit('.', 1)  
    if len(parts) == 2:
        return ".".join([f"{parts[0]}_before_feedback", parts[1]]) 
    return path  

def code_to_image2(code, img_save_path):
    import matplotlib.pyplot as plt
    import numpy as np
    import seaborn as sns
    exec_globals = {"plt": plt, "io": io, "np":np, 'sns':sns}
    exec_locals = {}
    print('Start Executing Code and Save Final Image')
    try:
        code_n = code.replace("plt.show()", f"plt.savefig('{img_save_path}')\nplt.close('all')")
        exec(code_n, exec_globals, exec_locals)
        message = "Save Image Successfully!"
        print(message)
        return code, True, None
    except Exception as e:
        message = f"Error during Save : {str(e)}"
        print(message)
        return code, False, str(e)

In [131]:
try_count = 0
while try_count < 3:
    print('Getting Code with Max 3 trials')
    response_text = await _call_openai_api(SYSTEM_PROMPT_CODE_GEN, user_content)
    # print(response_text)
    count_gpt4o_tokens(response_text)
    match = re.search(r'```python\n(.*?)```', response_text, flags=re.DOTALL)
    # print(match)
    if match:
        code = match.group(1).strip()
        # print(code)
        tmp_img_path = add_idx(img_save_path)
        print(tmp_img_path)
        code, log, error_message = code_to_image2(code, tmp_img_path)
        # print(log)
        if log:
            # print(code)
            print('Stop Before Max3')
            # return code, tmp_img_path, None, data_description
            break
        else:
            print(f'Try Again {try_count+1}')
            try_count += 1
    else:
        print(f'Try Again {try_count+1}')
        try_count += 1
print('No code unitl Max3')
# return code, None, error_message, data_description

Getting Code with Max 3 trials
Token count: 633
./50_before_feedback.png
Start Executing Code and Save Final Image
Error during Save : name 'np' is not defined
Try Again 1
Getting Code with Max 3 trials
Token count: 633
./50_before_feedback.png
Start Executing Code and Save Final Image
Error during Save : name 'np' is not defined
Try Again 2
Getting Code with Max 3 trials
Token count: 633
./50_before_feedback.png
Start Executing Code and Save Final Image
Error during Save : name 'np' is not defined
Try Again 3
No code unitl Max3


In [124]:
def _describe_data(data_path):
    if not data_path:
        return "No data file provided."
    try:
        data = pd.read_csv(data_path)
        description = {
            "data_path": data_path,
            "columns": list(data.columns),
            "dtypes": data.dtypes.apply(lambda x: x.name).to_dict(),
            "shape": data.shape,
            "sample": data.head(3).to_dict()
        }
        return str(description)
    except Exception as e:
        return f"Error reading data file: {e}"    

In [125]:
def add_idx(path):
    parts = path.rsplit('.', 1)  
    if len(parts) == 2:
        return ".".join([f"{parts[0]}_before_feedback", parts[1]]) 
    return path  

def code_to_image2(code, img_save_path):
    import matplotlib.pyplot as plt
    exec_globals = {"plt": plt, "io": io}
    exec_locals = {}
    print('Start Executing Code and Save Final Image')
    try:
        code_n = code.replace("plt.show()", f"plt.savefig('{img_save_path}')\nplt.close('all')")
        exec(code_n, exec_globals, exec_locals)
        message = "Save Image Successfully!"
        print(message)
        return code, True, None
    except Exception as e:
        message = f"Error during Save : {str(e)}"
        print(message)
        return code, False, str(e)

In [126]:
async def get_code_content(nl_query, data_path_list, img_save_path):
        
    if data_path_list:
        data_description_list = []
        for data_path in data_path_list:
            root_path = os.path.abspath(os.path.dirname(os.curdir))
            data_path = os.path.normpath(os.path.join(root_path, data_path))
            single_data_description = _describe_data(data_path)
            data_description_list.append(single_data_description)
        data_description = "[" + "], [".join(data_description_list) + "]"

    else :
        data_description='''There is no dataset provided.'''


    extended_query = await _query_extension(nl_query)
    
    user_content = f"""Detailed Instructions:
{extended_query}

Data Description:
Note that you must use the right csv data which is stated in "data_path". DO NOT MOCK DATA if data_path is provided!!!
If there are no data_path then just follow the Detailed Instructions. There might be an instruction about the data.
{data_description}

Please generate Python code using `matplotlib.pyplot` and 'seaborn' and 'pandas' to create the requested plot. Ensure the code is outputted within the Markdown format like ```python\n...```. 
You MUST follow the format and Don't savefig or save anything else.
"""
    
    try_count = 0
    while try_count < 3:
        print('Getting Code with Max 3 trials')
        response_text = await _call_openai_api(SYSTEM_PROMPT_CODE_GEN, user_content)
        # print(response_text)
        match = re.search(r'```python\n(.*?)```', response_text, flags=re.DOTALL)
        # print(match)
        if match:
            code = match.group(1).strip()
            # print(code)
            tmp_img_path = add_idx(img_save_path)
            code, log, error_message = code_to_image2(code, tmp_img_path)
            # print(log)
            if log:
                # print(code)
                print('Stop Before Max3')
                return code, tmp_img_path, None, data_description
            else:
                print(f'Try Again {try_count+1}')
                try_count += 1
        else:
            print(f'Try Again {try_count+1}')
            try_count += 1
    print('No code unitl Max3')
    return code, None, error_message, data_description


In [127]:
# Function to encode the image to base64 format
def encode_image(image_path):
    print('Encoding Image to Base64')
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [128]:
async def visual_feedback(ori_query, code, img_save_path, data_description):
    # logger.info('Starting Feedback...')
    # Encode the image to base64 for sending as a message
    try:
        base64_image = encode_image(img_save_path)
    except:
        base64_image = "No image found due to Error"
        
    # Prepare the messages for GPT-4o, including the system prompt, user prompt with code and query, and the image
    messages = [
        {"role": "system", 
        "content": '''Given a piece of code, a user query, and an image of the current plot, please determine whether the plot has faithfully followed the user query. Your task is to provide instruction to make sure the plot has strictly completed the requirements of the query. Please output a detailed step by step instruction on how to use python code to enhance the plot.'''},
        {"role": "user", 
        "content": f'''Here is the code: [Code]:
"""
{code}
"""

Here is the user query and the about the data description: [Query]:
"""
{ori_query}

{data_description}
"""

Carefully read and analyze the user query to understand the specific requirements. Examine the provided Python code to understand how the current plot is generated. Check if the code aligns with the user query in terms of data selection, plot type, and any specific customization. Look at the provided image of the plot. Assess the plot type, the data it represents, labels, titles, colors, and any other visual elements. Compare these elements with the requirements specified in the user query.
Note any differences between the user query requirements and the current plot. Based on the identified discrepancies, provide step-by-step instructions on how to modify the Python code to meet the user query requirements. Suggest improvements for better visualization practices, such as clarity, readability, and aesthetics, while ensuring the primary focus is on meeting the user's specified requirements.'''}
]
    count_gpt4o_tokens(messages[0]['content'])
    count_gpt4o_tokens(messages[-1]['content'])

    messages[-1]["content"] += f"\n\n![plot](data:image/png;base64,{base64_image})"
    # print(messages[0]['content'])
    # Call the completion function to get feedback from GPT-4
    
    feedback = await _call_openai_api(messages[0]['content'], messages[-1]['content'])
    # print(feedback)
    return feedback

In [132]:
tmp_img_path
data_description
user_query = query_list
initial_code = code

In [133]:
visual_feedback = await visual_feedback(user_query, initial_code, img_save_path = tmp_img_path, data_description=data_description)

Encoding Image to Base64
Token count: 69
Token count: 1073


In [134]:
count_gpt4o_tokens(visual_feedback)

Token count: 1471


In [135]:
async def feedback_aggregation(code, feedback, data_description):
    code_with_feedback = ""
    code_with_feedback += f"------\nData Description:{data_description}\nInitial code:\n{code}\nFeedback :\n{feedback}"
    return code_with_feedback

code_with_feedback = await feedback_aggregation(initial_code, visual_feedback, data_description)
count_gpt4o_tokens(code_with_feedback)

Token count: 2115


In [136]:
response_text = await _call_openai_api(SYSTEM_PROMPT_CODE_GEN, code_with_feedback)
count_gpt4o_tokens(response_text)
# print('#'*5, 'Final Code based on Feedback', '#' * 5)
# print(response_text)
match = re.search(r"```python(.*?)```", response_text, flags=re.DOTALL)

# match = re.search(r"```python(.*?)plt\.show\(\)", response_text, flags=re.DOTALL)
if match:
    pass
    # final_code
    # return match.group(1).strip(), prompt
    # print(code)
else:
    pass
    # return None, prompt

Token count: 645


In [137]:
# 실험 데이터
experiments = {
    "Query Expansion": {
        "input_tokens": [358, 133, 167, 208, 246],
        "output_tokens": [1340, 1323, 1660, 1065, 997],
    },
    "Code Generation": {
        "input_tokens": [1460, 1370, 1780, 1185, 1117],
        "output_tokens": [448, 1020, 1669, 1027, 1899],
    },
    "Visual Feedback": {
        "input_tokens": [1086, 740, 1013, 831, 1142],
        "output_tokens": [1264, 997, 1244, 1131, 1471],
    },
    "Final Code Generation": {
        "input_tokens": [1730, 1352, 1838, 1501, 2115],
        "output_tokens": [461, 478, 682, 375, 645],
    },
}

# 평균 계산 함수
def calculate_averages(experiments):
    averages = {}
    for stage, data in experiments.items():
        avg_input = sum(data["input_tokens"]) / len(data["input_tokens"])
        avg_output = sum(data["output_tokens"]) / len(data["output_tokens"])
        averages[stage] = {"avg_input_tokens": avg_input, "avg_output_tokens": avg_output}
    return averages

# 평균 계산 실행
average_tokens = calculate_averages(experiments)
average_tokens

{'Query Expansion': {'avg_input_tokens': 222.4, 'avg_output_tokens': 1277.0},
 'Code Generation': {'avg_input_tokens': 1382.4, 'avg_output_tokens': 1212.6},
 'Visual Feedback': {'avg_input_tokens': 962.4, 'avg_output_tokens': 1221.4},
 'Final Code Generation': {'avg_input_tokens': 1707.2,
  'avg_output_tokens': 528.2}}